# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker
!pip install s3fs"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n!pip install s3fs'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
TRAIN_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.4.4", "data_v0.4.4_train.csv")
VAL_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.4.4", "data_v0.4.4_val.csv")

train_df = pd.read_csv(TRAIN_PATH)
val_df = pd.read_csv(VAL_PATH)

## Sagemaker Prep

### Session

Configure SageMaker

In [7]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [8]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [9]:
sample = False  # To make the computations faster, sample = True.

if sample:
    train_df = train_df.sample(n=1000)
    val_df = val_df.sample(n=1000)
    
job_name = f"pytorch-{formatted_time()}-subpillars-model-test-mlflow"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


train_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
val_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [10]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

In [11]:
DEV_BUCKET

S3Path('s3://sagemaker-deep-experiments-dev')

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [12]:
from sagemaker.pytorch import PyTorch


hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-2d-subpilllars",
    'max_len': 128,
    'epochs': 3,
    'model_name': 'microsoft/Multilingual-MiniLM-L12-H384',
    'tokenizer_name': 'xlm-roberta-base',
    'dropout_rate': 0.3,
    'pred_threshold':0.5,
    'output_length': 384,
    'learning_rate': 5e-5,
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type='ml.p2.xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [13]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [14]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-07-28 07:59:29 Starting - Starting the training job...
2021-07-28 07:59:52 Starting - Launching requested ML instancesProfilerReport-1627459166: InProgress
...
2021-07-28 08:00:32 Starting - Preparing the instances for training........
2021-07-28 08:02:53 Downloading - Downloading input data
2021-07-28 08:02:53 Training - Downloading the training image.........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-07-28 08:07:50,048 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-07-28 08:07:50,073 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-07-28 08:07:53,110 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-07-28 08:07:53,641 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2021-07

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=0cbc8644c05fe810aabaf12ede6df3611b05e07bcfebf71b2945b22551019ddc
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=69750 sha256=995728fcba5ac0fa66fb6696cb686878369b9826dc29cb0d797ab861456e9b8a
  Stored in directory: /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3161 sha256=d9c412a267b424e9a3da7dd8f4e92b94309e7eaf32a5c2e881a7118561e2ee34
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.2-py3-none-any.whl size=17398 sha256=803ce680735ada510167bed0848454615b156c7c74e4accb6cfc7afc2bf0b4b3
  Stored in directory:

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-07-28 08:11:45.658 algo-1:79 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-07-28 08:11:45.730 algo-1:79 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-07-28 08:11:45.731 algo-1:79 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-07-28 08:11:45.731 algo-1:79 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-07-28 08:11:45.732 algo-1:79 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-07-28 08:11:45.732 algo-1:79 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-07-28 08:11:46.036 algo-1:79 INFO hook.py:591] name:model.l0.embeddings.word_embeddings.weight count_params:96014208
[2021-07-28 08:11:46.037 algo-1:79 INFO hook.py:591] nam

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/1547 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/1547 [00:00<?, ?it/s] #015Epoch 0:   2%|▏         | 30/1547 [00:16<13:37,  1.86it/s]#015Epoch 0:   2%|▏         | 30/1547 [00:16<13:37,  1.86it/s, loss=2.23, v_num=0, val_f1_epoch=0.0709, val_loss_epoch=0.709, train_f1=0.243]#015Epoch 0:   4%|▍         | 60/1547 [00:31<13:13,  1.88it/s, loss=2.23, v_num=0, val_f1_epoch=0.0709, val_loss_epoch=0.709, train_f1=0.243]#015Epoch 0:   4%|▍         | 60/1547 [00:31<13:13,  1.88it/s, loss=0.496, v_num=0, val_f1_epoch=0.0709, val_loss_epoch=0.709, train_f1=0.598]#015Epoch 0:   6%|▌         | 90/1547 [00:47<12:53,  1.88it/s, loss=0.496, v_num=0, val_f1_epoch=0.0709, val_loss_epoch=0.709, train_f1=0.598]#015Epoch 0:   6%|▌         | 90/1547 [00:47<12:53,  1.88it/s, loss=0.394, v_num=0, val_f1_epoch=0.0709, val_loss_epoch=0.709, train_f1=0.566]#015Epoch 0:   8%|▊ 

#015Validating: 100%|██████████| 172/172 [00:55<00:00,  3.08it/s]#033[A#015Epoch 0: 100%|██████████| 1547/1547 [13:05<00:00,  1.97it/s, loss=0.171, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.600, val_f1_step=0.607, val_loss_step=0.196]
#015                                                             #033[A#015Epoch 0: 100%|██████████| 1547/1547 [13:09<00:00,  1.96it/s, loss=0.171, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.600, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 0:   0%|          | 0/1547 [00:00<?, ?it/s, loss=0.171, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.600, val_f1_step=0.607, val_loss_step=0.196]           #015Epoch 1:   0%|          | 0/1547 [00:00<?, ?it/s, loss=0.171, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.600, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:   2%|▏         | 30/1547 [00:16<13:38,  1.85it/s, loss=0.171, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0

674, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:  83%|████████▎ | 1290/1547 [11:25<02:16,  1.88it/s, loss=0.139, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.674, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:  83%|████████▎ | 1290/1547 [11:25<02:16,  1.88it/s, loss=0.168, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.741, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:  85%|████████▌ | 1320/1547 [11:41<02:00,  1.88it/s, loss=0.168, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.741, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:  85%|████████▌ | 1320/1547 [11:41<02:00,  1.88it/s, loss=0.138, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.654, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:  87%|████████▋ | 1350/1547 [11:57<01:44,  1.88it/s, loss=0.138, v_num=0, val_f1_epoch=0.632, val_loss_epoch=0.187, train_f1=0.654, val_f1_step=0.607, val_loss_step=0.196]#015Epoch 1:  87%|████████▋ | 1350/1547 [

69, val_f1_step=0.720, val_loss_step=0.172]#015Epoch 2:  83%|████████▎ | 1290/1547 [11:25<02:16,  1.88it/s, loss=0.134, v_num=0, val_f1_epoch=0.721, val_loss_epoch=0.164, train_f1=0.769, val_f1_step=0.720, val_loss_step=0.172]#015Epoch 2:  83%|████████▎ | 1290/1547 [11:25<02:16,  1.88it/s, loss=0.133, v_num=0, val_f1_epoch=0.721, val_loss_epoch=0.164, train_f1=0.723, val_f1_step=0.720, val_loss_step=0.172]#015Epoch 2:  85%|████████▌ | 1320/1547 [11:41<02:00,  1.88it/s, loss=0.133, v_num=0, val_f1_epoch=0.721, val_loss_epoch=0.164, train_f1=0.723, val_f1_step=0.720, val_loss_step=0.172]#015Epoch 2:  85%|████████▌ | 1320/1547 [11:41<02:00,  1.88it/s, loss=0.132, v_num=0, val_f1_epoch=0.721, val_loss_epoch=0.164, train_f1=0.687, val_f1_step=0.720, val_loss_step=0.172]#015Epoch 2:  87%|████████▋ | 1350/1547 [11:57<01:44,  1.88it/s, loss=0.132, v_num=0, val_f1_epoch=0.721, val_loss_epoch=0.164, train_f1=0.687, val_f1_step=0.720, val_loss_step=0.172]#015Epoch 2:  87%|████████▋ | 1350/1547 [1


2021-07-28 08:53:47 Uploading - Uploading generated training model
2021-07-28 08:55:47 Completed - Training job completed
ProfilerReport-1627459166: IssuesFound
Training seconds: 3184
Billable seconds: 3184
